In [1]:
# last one - music
# use the same primary key as my other
# columns - project and creator pages

In [2]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [3]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [4]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW DATABASES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('information_schema',)
('kickscrape',)
('kickstarter',)
('mysql',)
('performance_schema',)
('test',)
('testdb',)
('world_innodb',)


In [74]:
project_url = \
'https://www.kickstarter.com/projects/1537415287/coil-iced-coffee-w-o-diluting-your-brew?ref=description'

project_request = requests.get(project_url)
project_raw = unidecode(project_request.text)

In [79]:
soup = BeautifulSoup(project_raw)
print soup.get_text()




  //<![CDATA[
    docElement = this.document.documentElement;
    docElement.className=docElement.className.replace(/\bno-js\b/,'') + 'js';
  //]]>
Coil: Iced Coffee w/o Diluting your Brew by Tyler Deeb — Kickstarter














































//<![CDATA[
(function(){window.formatters = window.formatters || {}; var formatting_options = {"distance_date_intervals":[{"min":5,"i18n_key":"schwamm.just_now"},{"min":29,"i18n_key":"schwamm.less_than_a_minute_ago"},{"min":2640,"i18n_key":"schwamm.m_minutes_ago","count_key":"m"},{"min":82800,"i18n_key":"schwamm.about_h_hours_ago","count_key":"h"},{"min":604800,"i18n_key":"schwamm.d_days_ago","count_key":"d"},{"min":15552000,"i18n_key":"schwamm.on_datetime","count_key":"t"},{"min":0,"i18n_key":"schwamm.on_year","count_key":"y"}],"currency_symbol_mappings":{"AFN":"[?]","ALL":"Lek","ARS":"$","EUR":"EUR","AUD":"$","AWG":"f","AZN":"man","BAM":"KM","BBD":"$","BGN":"lv","BMD":"$","BND":"$","BOB":"$b","BRL":"R$","BSD":"$","BWP":"P"

In [15]:
match = re.search('video',project_raw)
if match:
    print match.group()

video


In [16]:
# search all of my scraped pages for videos, find the earliest with a video by this critera.
# check for false negatives -- success/failure for these videos

In [18]:
n_campaigns = 191384
all_ids = range(1,n_campaigns+1)

In [26]:
has_video_df = pd.DataFrame(index=all_ids,columns=['Id','has_video','URL'])

In [27]:
has_video_df.tail()

,Id,has_video,URL
191380,NaN,NaN,NaN
191381,NaN,NaN,NaN
191382,NaN,NaN,NaN
191383,NaN,NaN,NaN
191384,NaN,NaN,NaN


In [34]:
with con:
    for primary_key in all_ids:
        
        if not primary_key%5000:
            print primary_key
            
        cur = con.cursor()
        cur.execute("SELECT Project_html FROM Scraped_pages WHERE Id = %s",primary_key)
        row = cur.fetchall()
        project_raw = row[0][0]
        cur.close()
        
        cur = con.cursor()
        cur.execute("SELECT URL FROM Project_info WHERE Id = %s",primary_key)
        row = cur.fetchall()
        url = row[0][0]
        cur.close()
        
        
        match = re.search('video',project_raw)
        if match:
            has_video = True
        else:
            has_video = False
        has_video_df.ix[primary_key,:] = [primary_key,has_video,url]
        
    

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000


In [40]:
has_video_df.head()

,has_video,URL,launch_t
1,True,https://www.kickstarter.com/projects/213558405...,0
2,True,https://www.kickstarter.com/projects/647561166...,0
3,True,https://www.kickstarter.com/projects/158735721...,0
4,True,https://www.kickstarter.com/projects/318653143...,0
5,True,https://www.kickstarter.com/projects/vermontco...,0


In [39]:
has_video_df['launch_t'] = 0

In [42]:
# append launch time (unix time)
with con:
    cur = con.cursor()
    for primary_key in all_ids:
        if not primary_key%5000:
            print primary_key

        cur.execute("SELECT Launch_time FROM Project_info WHERE Id = %s",primary_key)
        row = cur.fetchall()
        launch_t = row[0][0]
        has_video_df.ix[primary_key,'launch_t'] = launch_t
    cur.close()

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000


In [44]:
has_video_df.to_pickle('has_video_df.save')

In [50]:
has_video_df.ix[9020:9030,:]

,has_video,URL,launch_t
9020,True,https://www.kickstarter.com/projects/helix3d/h...,1382475326
9021,True,https://www.kickstarter.com/projects/usah2o/cu...,1352166872
9022,True,https://www.kickstarter.com/projects/unawe/uni...,1399626177
9023,True,https://www.kickstarter.com/projects/ryochijii...,1316331346
9024,False,https://www.kickstarter.com/projects/simonedav...,1300833819
9025,True,https://www.kickstarter.com/projects/154547164...,1371763554
9026,True,https://www.kickstarter.com/projects/openrail/...,1363645558
9027,False,https://www.kickstarter.com/projects/291941934...,1383425883
9028,True,https://www.kickstarter.com/projects/securly/o...,1413325051
9029,True,https://www.kickstarter.com/projects/astroonth...,1301801165


In [52]:
print has_video_df.ix[9027,'URL']

https://www.kickstarter.com/projects/291941934/software-ksp-launcher?ref=category


In [58]:
non_zero_launch_t_i = np.where(has_video_df['launch_t'] > 0)[0]

In [59]:
np.argmin(has_video_df.ix[non_zero_launch_t_i,'launch_t'])

7032

In [60]:
non_zero_launch_t_i[7032]

7033

In [62]:
has_video_df.ix[7033,'launch_t']

1368064360

In [63]:
import time
time.ctime(1368064360)

'Wed May  8 21:52:40 2013'

In [68]:
only_ts = has_video_df['launch_t']
only_ts.sort(inplace=True)

In [70]:
only_ts.head(20)

165001             0
75926              0
7032               0
190864             0
145812             0
142407             0
152250    1240340568
127314    1240438073
177500    1240602723
8072      1240673781
152445    1240834239
78266     1240919741
123008    1240963461
17594     1240966730
6791      1240972657
65439     1240975592
38013     1240980224
80978     1241005923
162432    1241035693
8534      1241039475
Name: launch_t, dtype: int64

In [71]:
time.ctime(1240340568)

'Tue Apr 21 15:02:48 2009'

In [72]:
# find the earliest campaign with a video, international shipping,
# reward dates, etc -- goal is to find a date range in which to restrict 
# my prediction.
# can I then use faq, rewards, etc? 
#
# add live campaigns -- check my text.
#
# then excluded cancelled and suspended campaigns
#
# then m